In [2]:
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
import pandas as pd

In [19]:
#.env 파일에서 opendata_key 키 설정
#goodyoung: 4e465a626d30363134304465454b78

In [10]:
# !pip3 install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [ ]:
# 1	INF_ID	서비스 ID
# 2	INF_NM	서비스명
# 3	CATE_NM	대분류
# 4	DITC_NM	중분류
# 5	MAP_CATE_NM	소분류
# 6	MNG_ORGAN_NAME	제공기관
# 7	MNG_STATION_NAME	제공부서명
# 8	LINK_DESC	시스템명
# 9	MANAGER_NAME	담당자명
# 10	MANAGER_PHONE	담당자연락처
# 11	CHNG_LOAD_NM	갱신주기
# 12	DATA_LT_NM	최종갱신일자
# 13	LINK_INFO	제공사이트
# 14	SRV_TYPE	제공형식
# 15	SHORT_URL	서비스URL

In [3]:
class GetData:
    def __init__(self):
        self.data = []
        self.datas = []
        self.setting()
        self.df_total = pd.DataFrame()
        
    def setting(self):
        load_dotenv()
        self.key = os.getenv("opendata_key")
        
    def test(self, api_num):
        for num in range(api_num):
            self.url = f'http://openAPI.seoul.go.kr:8088/{self.key}/xml/SearchCatalogService/{num}000/{num}999/'
            self.req = requests.get(self.url)
            self.soup = BeautifulSoup(self.req.text, 'xml')
            #더 이쁘게,,, 3중 포문이 나을까,,?
            self.id = self.soup.select('INF_ID') #서비스ID
            self.name = self.soup.select('INF_NM') #서비스명
            self.big = self.soup.select('CATE_NM') #대분류
            self.middle = self.soup.select('DITC_NM') #중분류
            self.small = self.soup.select('MAP_CATE_NM') #소분류
            self.agency  = self.soup.select('MNG_ORGAN_NAME')#제공기관
            self.department = self.soup.select('MNG_STATION_NAME') #제공부서명
            self.system = self.soup.select('LINK_DESC') #시스템명
            self.manager = self.soup.select('MANAGER_NAME') #담당자명
            self.manager_tel = self.soup.select('MANAGER_PHONE') #담당자연락처
            self.renewal = self.soup.select('CHNG_LOAD_NM') #갱신주기
            self.final_renewal = self.soup.select('DATA_LT_NM') #최종갱신일자
            self.site = self.soup.select('LINK_INFO') #제공사이트
            self.form = self.soup.select('SRV_TYPE') #제공형식
            self.service_url = self.soup.select('SHORT_URL') #서비스URL
            
            
            for s in range(len(self.name)):
                col = [self.id[s].text, self.name[s].text, self.big[s].text, self.middle[s].text, self.small[s].text,
                       self.agency[s].text, self.department[s].text, self.system[s].text, self.manager[s].text, self.manager_tel[s].text,
                      self.renewal[s].text, self.final_renewal[s].text, self.site[s].text, self.form[s].text, self.service_url[s].text]
                self.data.append(col)
                
    def test2(self):        
        for j in range(17):
            url = f'http://openAPI.seoul.go.kr:8088/{self.key}/xml/SearchOpenDataServiceList/{j}000/{j}999/'
            req = requests.get(url)
            soup = BeautifulSoup(req.text, 'xml')
            name = soup.select('INF_NM')
            exp = soup.select('INF_EXP')
            for k in range(len(name)):
                col = [name[k].text, exp[k].text]
                self.datas.append(col)    
            print('ads',j)

    def dataMerge(self):
        ##1
        df = pd.DataFrame(self.data, columns = ['서비스ID','서비스명','대분류','중분류','소분류','제공기관','제공부서명','시스템명','담당자명','담당자연락처',
                               '갱신주기','최종갱신일자','제공사이트','제공형식','서비스URL'])
        df.drop_duplicates(subset = '서비스명', inplace = True)
        print(f"서울시 개방현황: 총 {len(df)}개 입니다.")
        
        ##2
        df2 = pd.DataFrame(self.datas, columns=['서비스명','서비스설명'])
        df2.drop_duplicates(subset = '서비스명', inplace = True)
        print(f"서울시 서비스: 총 {len(df2)}개 입니다.")
        
        ##3
        self.df_total = pd.merge(df,df2,on = '서비스명')
        print(f"서울시 공공데이터: 총 {len(self.df_total)}개 입니다.")
        
        return self.df_total
    
    
    def changeurl(self,s):
        return f"https://data.seoul.go.kr/dataList/{s}/S/1/datasetView.do"
    
    
    def dataPreprocessing(self):
        self.df_total.drop(['서비스URL'], axis = 1, inplace = True)
        self.df_total['urls'] = self.df_total['서비스ID'].apply(self.changeurl)
        
    def getapi(self,api_num = 1): # api_num =8주면 보통 다 크롤링 해온다.
        self.test(api_num) # 서울시 개방현황
        self.test2() # 서울시 서비스 현황
        self.dataMerge() # 데이터 합치기
        self.dataPreprocessing() # url만들기
        return self.df_total

In [4]:
a = GetData()

In [5]:
s = a.getapi(api_num=8)

ads 0
ads 1
ads 2
ads 3
ads 4
ads 5
ads 6
ads 7
ads 8
ads 9
ads 10
ads 11
ads 12
ads 13
ads 14
ads 15
ads 16
서울시 개방현황: 총 7398개 입니다.
서울시 서비스: 총 7398개 입니다.
서울시 공공데이터: 총 7398개 입니다.


In [7]:
s.to_csv("최종인가봉가.csv", index = False)